In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import requests
import json
import os
import locale
import requests
%run 5_Funciones.ipynb
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
from datetime import datetime, timedelta

Primero, llamos a la función 'obtener_encabezado_api' del JN 4_Funciones, para obtener el encabezado de autorización AEMET utilizando un token almacenado en el archivo '../api.txt':

In [3]:
aemet = obtener_encabezado_api(api_file_path='../api.txt')

Se definen las columnas de interés para el análisis de datos climáticos:

In [9]:
columnas_interes= ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']

In [10]:
columnas_procesar= ['tmed', 'prec', 'sol', 'tmin', 'tmax']

In [12]:
'''
Defino la lista de años a procesar (desde 2014 hasta 2023)
'''
anos_a_procesar = list(range(2014, 2024))

In [13]:
lista_dataframes = []

'''
Creo la lista vacía 'lista_dataframes' y la lleno iterando a través de años y meses, 
generando y procesando DataFrames mensuales con la función 'procesar_datos_mes', si las 
URLs de datos son válidas.
'''

for ano in anos_a_procesar:
    
    for mes in range(1, 13):
        
        fecha_inicio = f'{ano}-{mes:02d}-01T00:00:00UTC'
        fecha_fin = f'{ano}-{mes:02d}-31T00:00:00UTC'
        url_datos = obtener_url_datos(fecha_inicio, fecha_fin)

        if url_datos:
            df_mes = procesar_datos_mes(url_datos, columnas_interes)
            lista_dataframes.append(df_mes)

Una vez obtenido los datos de la API y procesados, el siguiente paso es concatener todos los DataFrames en uno solo:

In [15]:
df_final = pd.concat(lista_dataframes, ignore_index=True)

In [17]:
data = df_final.copy()

In [18]:
data['Provincia'].value_counts()

Provincia
A coruña                 120
Pontevedra               120
Las palmas               120
Leon                     120
Lleida                   120
Lugo                     120
Madrid                   120
Malaga                   120
Murcia                   120
Navarra                  120
Ourense                  120
Palencia                 120
Salamanca                120
Albacete                 120
Segovia                  120
Sevilla                  120
Soria                    120
Sta. cruz de tenerife    120
Tarragona                120
Teruel                   120
Toledo                   120
Valencia                 120
Valladolid               120
Zamora                   120
La rioja                 120
Jaen                     120
Illes balears            120
Huesca                   120
Alicante                 120
Almeria                  120
Araba/alava              120
Asturias                 120
Avila                    120
Badajoz                  120
Barc

A continuación realizaré una corrección de nombres de provincias en la columna 'Provincia' del DataFrame 'data' mediante un diccionario de mapeo:

In [19]:
correccion_provincias = {'A coruña': 'A coruna',
                         'Las palmas':'Las palmas de gran canaria',
                         'Illes balears' : 'Islas baleares',
                        'Bizkaia': 'Vizcaya'}

data['Provincia'].replace(correccion_provincias, inplace=True)
data.head()

,Provincia,Periodo,Media_tmed,Media_prec,Media_sol,Media_tmin,Media_tmax
0,A coruna,2014-01,10.067727,8.375463,1.601290,7.825909,12.305909
1,Albacete,2014-01,7.683607,0.789655,4.319231,3.475410,11.872131
2,Alicante,2014-01,12.108871,0.263415,5.108602,7.168548,17.044355
3,Almeria,2014-01,12.734416,0.703401,6.237097,8.529870,16.937662
4,Araba/alava,2014-01,7.401613,4.291525,2.170968,3.866129,10.941935


Ya casi tengo el DataFrame de clima preparado, unicamente me falta añadir la latitud y longitud de cada provincia para poder hacer la representación visual correspondiente. Por lo que, una vez tratados los datos en el JP 'Webscraping_IGN' y exportado en CSV, lo llamo a continuación y realizo un join inner por Provincia:

In [26]:
lat_long= pd.read_csv('../data/IGN.csv')

In [38]:
df_merged = pd.merge(data, lat_long, on='Provincia', how='inner')
df_merged.head()

,Provincia,Periodo,Media_tmed,Media_prec,Media_sol,Media_tmin,Media_tmax,Población,Latitud,Longitud
0,A coruna,2014-01,10.067727,8.375463,1.601290,7.825909,12.305909,246047,43.370126,-8.391149
1,A coruna,2014-02,9.557014,8.672768,2.766443,6.594118,12.522624,246047,43.370126,-8.391149
2,A coruna,2014-03,11.561304,2.472889,5.590196,7.530870,15.593478,246047,43.370126,-8.391149
3,A coruna,2014-04,13.873171,2.612552,4.896732,10.282114,17.458537,246047,43.370126,-8.391149
4,A coruna,2014-05,14.216935,2.115447,7.304082,10.041532,18.396371,246047,43.370126,-8.391149


In [39]:
df_merged['Periodo'] = df_merged['Periodo'].dt.to_timestamp()

In [40]:
df_merged.dtypes

Provincia             object
Periodo       datetime64[ns]
Media_tmed           float64
Media_prec           float64
Media_sol            float64
Media_tmin           float64
Media_tmax           float64
Población              int64
Latitud              float64
Longitud             float64
dtype: object

In [41]:
df_merged.shape

(6000, 10)

In [42]:
df_merged.columns

Index(['Provincia', 'Periodo', 'Media_tmed', 'Media_prec', 'Media_sol',
       'Media_tmin', 'Media_tmax', 'Población', 'Latitud', 'Longitud'],
      dtype='object')

Por último, guardo el df_merged como clima:

In [43]:
df_merged.to_csv('../data/clima.csv', index=False)